#Tool for running NL-Bayes denoising on entire directories. This implementation ONLY works on Unix systems. If not using Colab, simply change the first cell to point at the comparable directories on your system. 

##Uses the Lebrun, Buades, and Morel c++ implementation found on IPOL 


## Converting images to png

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/My Drive/Lab/Grain-U-Net


In [3]:
#first, make save dirs

import os
from pathlib import Path
from pathlib import PurePath

test_dir = Path('data/test_Pt_Axon_5')
out_dir = Path('data/test_Pt_Axon_5_png')

os.mkdir(out_dir)

for FOV in test_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    
    new_dir = (out_dir / PurePath(FOV).name)
    if new_dir.is_dir():
        continue
    
    os.mkdir(new_dir)
    os.mkdir(new_dir / 'raw')
    

In [4]:
#reading in images and converting to png + cropping

from pathlib import Path
from skimage import io
import numpy as np

for FOV in test_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    for image in FOV.glob('raw/*.tiff'):
        img = io.imread(image)
        img = img[124:1924, 124:1924]
        save_path = out_dir / PurePath(FOV).name / 'raw' / (str(PurePath(image).name).split('.')[0] + '.png')
        io.imsave(save_path, (img))
        

## Running denoising

In [5]:
#more  save dirs

import os
from pathlib import Path
from pathlib import PurePath

noisy_dir = Path('data/test_Pt_Axon_5_png')
clean_dir = Path('data/test_Pt_Axon_5_denoised')

os.mkdir(clean_dir)

for FOV in noisy_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    
    new_dir = (clean_dir / PurePath(FOV).name)
    if new_dir.is_dir():
        continue
    
    os.mkdir(new_dir)
    os.mkdir(new_dir / 'raw')

In [6]:
#give needed permissions and compile code

!chmod +x ./src/nl-bayes_20130617/NL_Bayes
%cd src/nl-bayes_20130617
!make
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/MyDrive/Lab/Grain-U-Net/src/nl-bayes_20130617
make: 'NL_Bayes' is up to date.
/content/drive/My Drive/Lab/Grain-U-Net


In [7]:
#using subprocess to run the c++ code through command line. If a faster/more elegant/more accessible solution is desired, recommend a boost python implementation

import subprocess

subprocess.run(['sh', '-c', './src/nl-bayes_20130617/NL_Bayes'], capture_output=True)

noise = 30

for FOV in noisy_dir.glob('*'):
    if not (FOV / 'raw').is_dir(): 
        continue
    
    for image in FOV.glob('raw/*.png'):
        save_dir = clean_dir / PurePath(FOV).name / 'raw' / PurePath(image).name
        program = './src/nl-bayes_20130617/NL_Bayes'
        arg = ' {} {} 0 noisy.png {} basic.png difference.png bias.png biasbasic.png diffbias.png 1 0 0'.format(image, str(noise), save_dir)
        subprocess.run(['sh', '-c', program + arg])
        print('Finished image' + str(PurePath(image).name))

Finished image0.png
Finished image1.png
Finished image2.png
Finished image3.png
Finished image4.png
Finished image260.png
Finished image261.png
Finished image262.png
Finished image263.png
Finished image264.png
Finished image520.png
Finished image521.png
Finished image522.png
Finished image523.png
Finished image524.png
Finished image780.png
Finished image781.png
Finished image782.png
Finished image783.png
Finished image784.png
Finished image1040.png
Finished image1041.png
Finished image1042.png
Finished image1043.png
Finished image1044.png
Finished image1300.png
Finished image1301.png
Finished image1302.png
Finished image1303.png
Finished image1304.png
Finished image1560.png
Finished image1561.png
Finished image1562.png
Finished image1563.png
Finished image1564.png
Finished image1820.png
Finished image1821.png
Finished image1822.png
Finished image1823.png
Finished image1824.png
Finished image2080.png
Finished image2081.png
Finished image2082.png
Finished image2083.png
Finished image2084

In [8]:
!rm basic.png
!rm difference.png
!rm mea

rm: cannot remove 'mea': No such file or directory
